In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

from keras.layers import Activation, Conv2D, Conv3D, Dense, Dropout, Flatten, MaxPooling3D, GlobalMaxPool3D, GlobalMaxPool2D, ConvLSTM2D, TimeDistributed
# from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
# from keras.utils import np_utils
# from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split

from data_gen import ActionDataGenerator
from config import Config

from cnn3d_model import model3d, basic_model
from fundamental_model import simple_model

Using TensorFlow backend.


In [2]:
root_data_path='G:\\video_data\\small_activity_file\\csv_files\\'
classes = 10
data_gen_obj=ActionDataGenerator(root_data_path,temporal_stride=16,temporal_length=16,max_sample=20)

In [3]:
print(type(data_gen_obj))

<class 'data_gen.ActionDataGenerator'>


In [4]:
train_data = data_gen_obj.load_samples(data_cat='train')
val_data = data_gen_obj.load_samples(data_cat='val')
test_data = data_gen_obj.load_samples(data_cat='test')

the exception:  
end of data generator
the exception:  
end of data generator
the exception:  
end of data generator


In [5]:
print('num of train_samples: {}'.format(len(train_data)))
print('num of val_samples: {}'.format(len(val_data)))
print('num of test_samples: {}'.format(len(test_data)))

num of train_samples: 22265
num of val_samples: 2793
num of test_samples: 3192


In [6]:
train_generator = data_gen_obj.data_generator(train_data,batch_size=5,shuffle=True,n_classes=classes)
val_generator = data_gen_obj.data_generator(val_data, batch_size=5, shuffle=True,n_classes=classes)
test_generator = data_gen_obj.data_generator(test_data,batch_size=5,shuffle=True,n_classes=classes)

In [7]:
model = model3d(n_classes=10)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16, 192, 256, 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 16, 96, 128,  12032       input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 16, 96, 128,  0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling3d_1 (MaxPooling3D)  (None, 8, 48, 64, 32 0           re_lu_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
# Fit model using generator
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(train_generator, epochs=5, steps_per_epoch=len(train_data), validation_data=val_generator, validation_steps=len(val_data), verbose=1)

Epoch 1/5
 8169/22265 [==========>...................] - ETA: 1:39:16 - loss: 0.3354 - accuracy: 0.8924

In [ ]:
from utility import shows_plot
show_plot(hist.history, 'loss', 'val_loss')
show_plot(hist.history, 'accuracy', 'val_accuracy')

In [ ]:
score = model.evaluate(test_generator, steps=len(test_data))
print(score)

In [7]:
def get_model(num_classes=Config.num_classes):
    # Define model
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), input_shape=(16,192,256,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
    model.add(Dropout(0.25))
    '''
    model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
    model.add(Dropout(0.25))
    '''
    model.add(GlobalMaxPool3D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])
    model.summary()
    #plot_model(model, show_shapes=True,
    #           to_file='model.png')
    return model

In [26]:
def get_lstm_model(num_classes=Config.num_classes):
    # Define model
    model = Sequential()
    model.add(TimeDistributed( 
        Conv2D(128, (7, 7),
            padding='same', strides=(2, 2), activation='relu'), input_shape=(16,192,256,3)
    ))
    model.add(TimeDistributed( 
        Conv2D(64, (3, 3),
            padding='same', strides=(2,2), activation='relu')
    ))
    model.add(TimeDistributed( 
        Conv2D(32, (3,3),
            padding='same', strides=(2,2), activation='relu')
    ))
    model.add(ConvLSTM2D(64, kernel_size=(1,1), padding='same', return_sequences=True))
    model.add(TimeDistributed(GlobalMaxPool2D()))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])
    model.summary()
    #plot_model(model, show_shapes=True,
    #           to_file='model.png')
    return model

In [27]:
model = get_lstm_model(50)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 16, 96, 128, 128)  18944     
_________________________________________________________________
time_distributed_20 (TimeDis (None, 16, 48, 64, 64)    73792     
_________________________________________________________________
time_distributed_21 (TimeDis (None, 16, 24, 32, 32)    18464     
_________________________________________________________________
conv_lst_m2d_6 (ConvLSTM2D)  (None, 16, 24, 32, 64)    24832     
_________________________________________________________________
time_distributed_22 (TimeDis (None, 16, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)              

In [28]:
# Fit model using generator
hist = model.fit_generator(train_generator, epochs=3, steps_per_epoch=len(train_data))

Epoch 1/3
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
    27/124769 [..............................] - ETA: 7:59:36 - loss: 3.9301 - accuracy: 0.0000e+00(2, 16, 192, 256, 3
(2, 16, 192, 256, 3)
    67/124769 [..............................] - ETA: 6:27:46 - loss: 3.9218 - accuracy: 0.0075(2, 16, 192, 256, 3)


   137/124769 [..............................] - ETA: 5:56:13 - loss: 3.9055 - accuracy: 0.0219(2, 16, 192, 256, 3)


   184/124769 [..............................] - ETA: 5:48:26 - loss: 3.8963 - accuracy: 0.0245(2, 16, 192, 256, 3
(2, 16, 192, 256, 3)
   208/124769 [..............................] - ETA: 5:45:36 - loss: 3.8976 - accuracy: 0.0240(2, 16, 192, 256, 3)


   210/124769 [..............................] - ETA: 5:45:22 - loss: 3.8969 - accuracy: 0.0238(2, 16, 192, 256, 3
(2, 16, 192, 256, 3)
   279/124769 [..............................] - ETA: 5:40:07 - loss: 3.8878 - accuracy: 0.0323(2, 16, 192, 256, 3)


   349/124769 [..............................] - ETA: 5:36:55 - loss: 3.8755 - accuracy: 0.0330(2, 16, 192, 256, 3)


...

 21596/124769 [====>.........................] - ETA: 15:05:55 - loss: 3.1534 - accuracy: 0.1760(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
 21613/124769 [====>.........................] - ETA: 15:05:41 - loss: 3.1529 - accuracy: 0.1762(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
 21658/124769 [====>.........................] - ETA: 15:04:54 - loss: 3.1514 - accuracy: 0.1765(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
 21660/124769 [====>.........................] - ETA: 15:04:50 - loss: 3.1514 - accuracy: 0.1765(2, 16, 192, 256, 3)


 21693/124769 [====>.........................] - ETA: 15:05:28 - loss: 3.1502 - accuracy: 0.1768(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
 21731/124769 [====>.........................] - ETA: 15:04:56 - loss: 3.1488 - accuracy: 0.1771(2, 16, 192, 256, 3)


 21801/124769 [====>.........................] - ETA: 15:04:09 - loss: 3.1462 - accuracy: 0.1778(2, 16, 192, 256, 3)


 21834/124769 [====>.........................] - ETA: 15:04:26 - loss: 3.1448 - accuracy: 0.1780(2, 16, 192, 256, 3)
(2, 16, 192, 256, 3)
 21852/124769 [====>.........................] - ETA: 15:04:06 - loss: 3.1444 - accuracy: 0.1781(2, 16, 192, 256, 3)


KeyboardInterrupt: 